In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import category_encoders as ce
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


from imblearn.metrics import classification_report_imbalanced
from sklearn import metrics
from sklearn.metrics import roc_auc_score

import imblearn
import warnings

warnings.filterwarnings('ignore')

In [ ]:
#df = pd.read_csv('/kaggle/input/diabetic-patients-readmission-prediction/diabetic_data.csv')
df = pd.read_csv('diabetic_data.csv')
print(df.shape)
df.head()

In [ ]:
len(set(df.patient_nbr))

# Initial depuration and label encoding

In [ ]:
def remove_outliers(df, out_cols, T=1.5, verbose=True):
    # Copy of df
    new_df = df.copy()
    init_shape = new_df.shape
    # For each column
    for c in out_cols:
        q1 = new_df[c].quantile(.25)
        q3 = new_df[c].quantile(.75)
        col_iqr = q3 - q1
        col_max = q3 + T * col_iqr
        col_min = q1 - T * col_iqr
        # Filter data without outliers and ignoring nan
        filtered_df = new_df[(new_df[c] <= col_max) & (new_df[c] >= col_min)]
        if verbose:
            n_out = new_df.shape[0] - filtered_df.shape[0] 
            print(f" Columns {c} had {n_out} outliers removed")
        new_df = filtered_df
            
    if verbose:
        # Print shrink percentage
        lines_red = df.shape[0] - new_df.shape[0]
        print(f"Data reduced by {lines_red} lines, or {lines_red/df.shape[0]*100:.2f} %")
    return new_df

In [ ]:
#Data Cleaning
df = df.drop(columns=['weight', 'payer_code', 'medical_specialty','admission_type_id','discharge_disposition_id','admission_source_id'])
df = df.dropna()
df = df[df.race != '?']
df = df[df.diag_1 != '?']
df = df[df.diag_2 != '?']
df = df[df.diag_3 != '?']
df = df[df.gender != 'Unknown/Invalid']
df = df[df.number_outpatient < 17]
df = df[df.number_emergency < 14]
df = df[df.number_inpatient < 14]
df = df[df.number_diagnoses < 10]
df = df[df.number_diagnoses < 10]
df.number_outpatient = df.number_outpatient.mask(df.number_outpatient > 7, 8)
df.number_emergency = df.number_emergency.mask(df.number_emergency > 5, 6)
df.number_inpatient = df.number_inpatient.mask(df.number_inpatient > 9, 10)
df = remove_outliers(df, ['num_lab_procedures','num_medications'])

In [ ]:
#Labeling
diseases = ['metformin', 'repaglinide', 'nateglinide',
'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
'insulin', 'glyburide-metformin', 'glipizide-metformin',
'glimepiride-pioglitazone', 'metformin-rosiglitazone',
'metformin-pioglitazone', 'change']

df = df.replace({'readmitted' : { 'NO' : 0, '<30' : 2, '>30' : 1}}) #output feacture
df = df.replace({'diabetesMed' : { 'No' : 0, 'Yes' : 1}})
df = df.replace({'max_glu_serum' : { 'None' : 0, 'Norm' : 1, '>200' : 2, '>300' : 3}})
df = df.replace({'A1Cresult' : { 'None' : 0, 'Norm' : 1, '>7' : 2, '>8' : 3}})
df = df.replace({'gender' : { 'Female' : 0, 'Male' : 1}})
df = df.replace({'race' : { 'AfricanAmerican' : 0, 'Asian' : 1, 'Caucasian' : 2, 'Hispanic' : 3, 'Other' : 4}})
df = df.replace({'age' : { 'Female' : 0, 'Male' : 1}})
df = df.replace({'age' : { '[0-10)' : 0, '[10-20)' : 1, '[20-30)' : 2, '[30-40)' : 3, '[40-50)' : 4,
                           '[50-60)' : 5, '[60-70)' : 6, '[70-80)' : 7, '[80-90)' : 8, '[90-100)' : 9}})

opts = {'No': 0, 'Down': 1, 'Steady': 1, 'Up': 1}
opts1 = {'No': 0, 'Steady': 1}
opts2 = {'No': 0}
opts3 = {'No': 0, 'Steady': 1, 'Up': 1}
opts4 = {'No': 0, 'Ch' : 1}

encoding = []

for i in diseases:
    if set(df[i]) == opts.keys(): labeling = {i : opts} 
    elif set(df[i]) == opts1.keys(): labeling = {i : opts1} 
    elif set(df[i]) == opts2.keys(): labeling = {i : opts2}
    elif set(df[i]) == opts3.keys(): labeling = {i : opts3} 
    elif set(df[i]) == opts4.keys(): labeling = {i : opts4} 
    df = df.replace(labeling)
    encoding.append([labeling])
    
df['minor_diabetes_med'] = df.chlorpropamide + df.tolbutamide + df.miglitol+ df.troglitazone+ df.tolazamide+ df['glipizide-metformin']+ df['glimepiride-pioglitazone']+ df['metformin-rosiglitazone']

In [ ]:
def diagTreatmeant(dataframe):
    exclude = {'428':19 ,'414':20 ,'786':21 ,'486':22 ,'410':23 ,'427':24 ,'491':25, '715':26 ,'434':26 ,'780':27 ,'682':28 ,'996':28 ,'276':29 ,'250.8':30}
    newData = []
    for i in dataframe:
        if i.find('V') == -1 and i.find('E') == -1:
            tmp = float(i)
            if not i in exclude.keys():
                if tmp >= 1 and tmp <= 139:
                    newData.append(1)
                elif tmp >= 140 and tmp <= 239:
                    newData.append(2)
                elif tmp >= 240 and tmp <= 279:
                    newData.append(3)
                elif tmp >= 280 and tmp <= 289:
                    newData.append(4)
                elif tmp >= 290 and tmp <= 319:
                    newData.append(5)
                elif tmp >= 320 and tmp <= 389:
                    newData.append(6)
                elif tmp >= 390 and tmp <= 459:
                    newData.append(7)
                elif tmp >= 460 and tmp <= 519:
                    newData.append(8)
                elif tmp >= 520 and tmp <= 579:
                    newData.append(9)
                elif tmp >= 580 and tmp <= 629:
                    newData.append(10)
                elif tmp >= 630 and tmp <= 679:
                    newData.append(11)
                elif tmp >= 680 and tmp <= 709:
                    newData.append(12)
                elif tmp >= 710 and tmp <= 739:
                    newData.append(13)
                elif tmp >= 740 and tmp <= 759:
                    newData.append(14)
                elif tmp >= 760 and tmp <= 779:
                    newData.append(15)
                elif tmp >= 780 and tmp <= 799:
                    newData.append(16)
                else:
                    newData.append(17)
            else:
                newData.append(exclude[i])
        else:
            if i.find('V') != -1:
                newData.append(18)
            else:
                newData.append(31)
    return newData

# Data processing

In [ ]:
#Split output feacture
data = df.copy()
data = data.drop_duplicates(subset=['patient_nbr'])
data = data.drop(columns=['encounter_id', 'patient_nbr'])
y = pd.DataFrame(data.readmitted.copy())
data = data.drop(columns=['readmitted'])
data.columns

In [ ]:
#Diagnostic gouping accord to ICD9
data.diag_1 = diagTreatmeant(data.diag_1)
data.diag_2 = diagTreatmeant(data.diag_2)
data.diag_3 = diagTreatmeant(data.diag_3)
data = data.drop(columns=['acetohexamide', 'examide', 'citoglipton', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'chlorpropamide','tolbutamide','miglitol','troglitazone','tolazamide','glipizide-metformin','glimepiride-pioglitazone'])
ordinal_features = ['diag_1','diag_2', 'diag_3', 'race', 'age']
ce_ord = ce.TargetEncoder(cols = ordinal_features)
data = ce_ord.fit_transform(data, y)
ce_by = ce.BinaryEncoder(cols = diseases)
data = ce_ord.fit_transform(data, y)

In [ ]:
#Numeric data scale
numeric_feactures = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']
scaler = MinMaxScaler()
data[numeric_feactures] = scaler.fit_transform(data[numeric_feactures])

In [ ]:
#Feacture correlation
plt.figure(figsize=(30, 20))
sns.heatmap(data.corr(method = 'kendall'), annot = data.corr(), square = True)

In [ ]:
data['readmitted'] = y 

### No readmitted, 30< readmitted model 

In [ ]:
process_data = data.copy()
process_data = process_data[process_data.readmitted != 1]
process_data = process_data.replace({'readmitted' : { 2 : 1}}) #output feacture
y = pd.DataFrame(process_data.readmitted.copy())
process_data = process_data.drop(columns=['readmitted'])
#Label 0: No readmitted
#Label 1: 30< readmitted
y.value_counts().plot.bar(xlabel=['No readmitted','30< readmitted'])

In [ ]:
#Train test scolumnst
def train_test_split_generator(data, y, smote):   
    x_train, x_test, y_train, y_test = train_test_split(data, y ,test_size = 0.2, random_state=42, stratify = y)
    
    if (smote):
        sm = BorderlineSMOTE(random_state=42,k_neighbors=3)
        x_train, y_train = sm.fit_resample(x_train, y_train)
    
    print(y_train.value_counts().sort_index())
    print(y_test.value_counts().sort_index())
    
    x_train = x_train.astype('float64')
    y_train = y_train.astype('int')

    x_train=x_train.values
    y_train=y_train.values.reshape((-1))


    x_test = x_test.astype('float64')
    y_test = y_test.astype('int')

    x_test=x_test.values
    y_test=y_test.values.reshape((-1))
    
    return x_train, x_test, y_train, y_test
    
x_train, x_test, y_train, y_test = train_test_split_generator(process_data, y, False)

In [ ]:
#Class weight calculation
class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train),y_train))) 
sample_weights = [class_weights[y] for y in y_train]
train_sample_weight = sample_weights
print(class_weights)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#RandomForestClassifier
param_grid = {
    'max_depth': [6, 10, 17, 40],
    'n_estimators': [50, 100, 200, 300, 1000]
}
clf = RandomForestClassifier(class_weight=class_weights)
param_grid = GridSearchCV(estimator = clf, scoring = 'roc_auc_ovr', param_grid = param_grid, cv = StratifiedKFold(n_splits=5), verbose=4, n_jobs = -1)
param_grid.fit(x_train, y_train)
param_grid.best_params_
#6-300

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# GradientBoostingClassifier
param_grid = {
    'n_estimators': [100, 300, 1000],
    'learning_rate' : [0.01, 0.1, 0.001]
}

clf = GradientBoostingClassifier()
grid_search = GridSearchCV(estimator = clf, scoring = 'roc_auc_ovo', param_grid = param_grid, cv = StratifiedKFold(n_splits=5), verbose=10, n_jobs = -1)
grid_search.fit(x_train, y_train)
grid_search.best_params_

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# GradientBoostingClassifier
param_grid = {
    'reg_param': [0.0001, 0.001,0.01, 0.1], 
    'store_covariance': [True, False],
    'tol': [0.0001, 0.001,0.01, 0.1], 
}

clf = QuadraticDiscriminantAnalysis()
grid_search = GridSearchCV(estimator = clf, scoring = 'roc_auc_ovo', param_grid = param_grid, cv = StratifiedKFold(n_splits=5), verbose=10, n_jobs = -1)
grid_search.fit(x_train, y_train)
grid_search.best_params_

In [ ]:
from sklearn.svm import SVC

# GradientBoostingClassifier
param_grid = {
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001],
    'kernel': ['rbf']
}

clf = SVC()
grid_search = GridSearchCV(estimator = clf, scoring = 'roc_auc_ovo', param_grid = param_grid, cv = StratifiedKFold(n_splits=5), verbose=10, n_jobs = -1)
grid_search.fit(x_train, y_train)
grid_search.best_params_

In [ ]:
import keras
from keras.optimizers import adam_v2
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.constraints import max_norm


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=8, min_lr=1e-6,verbose = 1)
custom_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=20
)

METRICS = [ 
      keras.metrics.AUC(name='auc'),
]

d1 = 0.15


#model definition
def create_model(learn_rate):
    model = keras.Sequential([
    keras.layers.Dense(100, activation=tf.nn.selu, kernel_initializer = initializers.he_normal,bias_initializer=initializers.he_normal, input_shape=(x_train.shape[1],)),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
    
    keras.layers.Dense(90, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(80, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(70, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(60, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(50, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(40, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(30, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(20, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(10, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
 keras.layers.Dense(1, activation=  'sigmoid')
 ])
    optimizer = adam_v2.Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', metrics=[METRICS], optimizer=optimizer)
    return model


# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

max_n = 4.
num_folds = 5

# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
confusions_matrix = []
for train, test in kfold.split(inputs, targets):
      # Compile the model
    model = create_model(0.1)
    
    X_train_res, y_train_res = inputs[train], targets[train]
    X_test_res, y_test_res = inputs[test], targets[test]
        
      # Generate a print
    print('-----------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model , class_weight=class_weight
    history = model.fit(X_train_res, y_train_res, class_weight=class_weights, validation_data=(X_test_res, y_test_res),batch_size=256, epochs=100,callbacks=[custom_early_stopping, reduce_lr])

    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    fig, axs = plt.subplots(2)
    fig.suptitle('model loss')
    fig.legend(['train', 'val'], loc='upper left')

    axs[0].plot(history.history['loss'])
    axs[0].plot(history.history['val_loss'])
    axs[1].plot(history.history['auc'])
    axs[1].plot(history.history['val_auc'])

    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

      # Increase fold number
    fold_no = fold_no + 1

## Predict

In [ ]:
clf_Q1 = QuadraticDiscriminantAnalysis(reg_param=0.0001, store_covariance=True, tol=0.0001).fit(x_train, y_train)
clf_S1 = SVC(C=0.1, gamma=1, kernel='rbf').fit(x_train, y_train, sample_weight=train_sample_weight)
clf_G1 = GradientBoostingClassifier(learning_rate=0.01, n_estimators=1000).fit(x_train, y_train)
clf_R1 = RandomForestClassifier(max_depth=6, n_estimators=300,class_weight=class_weights).fit(x_train, y_train)

In [ ]:
# Predict QuadraticDiscriminantAnalysis
y_1 = clf_Q1.predict(x_test)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict SVC
y_1 = clf_S1.predict(x_test)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict GradientBoostingClassifier
y_1 = clf_G1.predict(x_test)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict RandomForestClassifier
y_1 = clf_R1.predict(x_test)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict
y_pred = model.predict(x_test)
y_1 = np.round(y_pred)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

## Predict with SMOTE

In [ ]:
x_train, x_test, y_train, y_test = train_test_split_generator(process_data, y, True)

In [ ]:
# Predict QuadraticDiscriminantAnalysis
clf_Q1 = QuadraticDiscriminantAnalysis(reg_param=0.0001, store_covariance=True, tol=0.0001).fit(x_train, y_train)

y_1 = clf_Q1.predict(x_test)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict SVC
clf_S1 = SVC(C=0.1, gamma=1, kernel='rbf').fit(x_train, y_train)

y_1 = clf_S1.predict(x_test)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict GradientBoostingClassifier
clf_G1 = GradientBoostingClassifier(learning_rate=0.01, n_estimators=1000).fit(x_train, y_train)

y_1 = clf_G1.predict(x_test)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict RandomForestClassifier
clf_R1 = RandomForestClassifier(max_depth=6, n_estimators=300).fit(x_train, y_train)
y_1 = clf_R1.predict(x_test)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
import keras
from keras.optimizers import adam_v2
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.constraints import max_norm


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=8, min_lr=1e-6,verbose = 1)
custom_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=20
)

METRICS = [ 
      keras.metrics.AUC(name='auc'),
]

d1 = 0.15


#model definition
def create_model(learn_rate):
    model = keras.Sequential([
    keras.layers.Dense(100, activation=tf.nn.selu, kernel_initializer = initializers.he_normal,bias_initializer=initializers.he_normal, input_shape=(x_train.shape[1],)),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(80, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(70, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(50, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(30, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(10, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
 keras.layers.Dense(1, activation=  'sigmoid')
 ])
    optimizer = adam_v2.Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', metrics=[METRICS], optimizer=optimizer)
    return model


# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

max_n = 4.
num_folds = 5

# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
confusions_matrix = []
for train, test in kfold.split(inputs, targets):
      # Compile the model
    model = create_model(0.1)
    
    X_train_res, y_train_res = inputs[train], targets[train]
    X_test_res, y_test_res = inputs[test], targets[test]
        
      # Generate a print
    print('-----------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model , class_weight=class_weight
    history = model.fit(X_train_res, y_train_res, validation_data=(X_test_res, y_test_res),batch_size=256, epochs=100,callbacks=[custom_early_stopping, reduce_lr])

    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    fig, axs = plt.subplots(2)
    fig.suptitle('model loss')
    fig.legend(['train', 'val'], loc='upper left')

    axs[0].plot(history.history['loss'])
    axs[0].plot(history.history['val_loss'])
    axs[1].plot(history.history['auc'])
    axs[1].plot(history.history['val_auc'])

    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

      # Increase fold number
    fold_no = fold_no + 1

In [ ]:
# Predict Neural network
y_pred = model.predict(x_test)
y_1 = np.round(y_pred)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

## Feacture selection

In [ ]:
process_data = data.copy()
process_data = process_data[process_data.readmitted != 1]
process_data = process_data.replace({'readmitted' : { 2 : 1}}) #output feacture
y = pd.DataFrame(process_data.readmitted.copy())
process_data = process_data.drop(columns=['readmitted'])
x_train, x_test, y_train, y_test = train_test_split_generator(process_data, y, False)

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
clf_R1 = RandomForestClassifier(max_depth=6, n_estimators=300,class_weight=class_weights)
sfs = SequentialFeatureSelector(clf_R1, direction='backward', scoring='roc_auc_ovo')
sfs.fit(x_train, y_train)

In [ ]:
sfs.get_support()

In [ ]:
x_train_red = sfs.fit_transform(x_train)
x_test_red = sfs.fit_transform(x_test)

In [ ]:
clf_S1 = SVC(C=0.1, gamma=1, kernel='rbf').fit(x_train_red, y_train, sample_weight=train_sample_weight)
clf_R1 = RandomForestClassifier(max_depth=6, n_estimators=300,class_weight=class_weights).fit(x_train_red, y_train)

In [ ]:
# Predict SVC
y_1 = clf_S1.predict(x_test_red)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict RandomForestClassifier
y_1 = clf_R1.predict(x_test_red)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
import keras
from keras.optimizers import adam_v2
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.constraints import max_norm


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=8, min_lr=1e-6,verbose = 1)
custom_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=20
)

METRICS = [ 
      keras.metrics.AUC(name='auc'),
]

d1 = 0.15


#model definition
def create_model(learn_rate):
    model = keras.Sequential([
    keras.layers.Dense(100, activation=tf.nn.selu, kernel_initializer = initializers.he_normal,bias_initializer=initializers.he_normal, input_shape=(x_train.shape[1],)),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
    
    keras.layers.Dense(90, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(80, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(70, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(60, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(50, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(40, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(30, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(20, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(10, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
 keras.layers.Dense(1, activation=  'sigmoid')
 ])
    optimizer = adam_v2.Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', metrics=[METRICS], optimizer=optimizer)
    return model

x_train = x_train_red
x_test = x_test_red

# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

max_n = 4.
num_folds = 5

# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
confusions_matrix = []
for train, test in kfold.split(inputs, targets):
      # Compile the model
    model = create_model(0.1)
    
    X_train_res, y_train_res = inputs[train], targets[train]
    X_test_res, y_test_res = inputs[test], targets[test]
        
      # Generate a print
    print('-----------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model , class_weight=class_weight
    history = model.fit(X_train_res, y_train_res, class_weight=class_weights, validation_data=(X_test_res, y_test_res),batch_size=256, epochs=100,callbacks=[custom_early_stopping, reduce_lr])

    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    fig, axs = plt.subplots(2)
    fig.suptitle('model loss')
    fig.legend(['train', 'val'], loc='upper left')

    axs[0].plot(history.history['loss'])
    axs[0].plot(history.history['val_loss'])
    axs[1].plot(history.history['auc'])
    axs[1].plot(history.history['val_auc'])

    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

      # Increase fold number
    fold_no = fold_no + 1

In [ ]:
# Predict Neural network
y_pred = model.predict(x_test_red)
y_1 = np.round(y_pred)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

## PCA

In [ ]:
process_data = data.copy()
process_data = process_data[process_data.readmitted != 1]
process_data = process_data.replace({'readmitted' : { 2 : 1}}) #output feacture
y = pd.DataFrame(process_data.readmitted.copy())
process_data = process_data.drop(columns=['readmitted'])
x_train, x_test, y_train, y_test = train_test_split_generator(process_data, y, False)

In [ ]:
from sklearn.decomposition import PCA

pca_test = PCA()
pca_test.fit(x_train)
sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=20, ymin=0, ymax=1)
display(plt.show())
evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.head(20))

In [ ]:
pca = PCA(n_components=20)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
clf_S1 = SVC(C=0.1, gamma=1, kernel='rbf').fit(x_train_pca, y_train, sample_weight=train_sample_weight)
clf_R1 = RandomForestClassifier(max_depth=6, n_estimators=300,class_weight=class_weights).fit(x_train_pca, y_train)

In [ ]:
# Predict SVC
y_1 = clf_S1.predict(x_test_pca)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Predict RandomForestClassifier
y_1 = clf_R1.predict(x_test_pca)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
import keras
from keras.optimizers import adam_v2
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.constraints import max_norm


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=8, min_lr=1e-6,verbose = 1)
custom_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=20
)

METRICS = [ 
      keras.metrics.AUC(name='auc'),
]

d1 = 0.15


#model definition
def create_model(learn_rate):
    model = keras.Sequential([
    keras.layers.Dense(100, activation=tf.nn.selu, kernel_initializer = initializers.he_normal,bias_initializer=initializers.he_normal, input_shape=(x_train.shape[1],)),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
    
    keras.layers.Dense(90, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(80, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(70, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(60, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.Dropout(d1),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(50, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(40, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(30, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(20, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
    keras.layers.Dense(10, activation=tf.nn.selu, kernel_constraint=max_norm(max_n), kernel_initializer = initializers.he_normal),
        keras.layers.BatchNormalization(),
        
 keras.layers.Dense(1, activation=  'sigmoid')
 ])
    optimizer = adam_v2.Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', metrics=[METRICS], optimizer=optimizer)
    return model

x_train = x_train_pca
x_test = x_test_pca

# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

max_n = 4.
num_folds = 5

# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
confusions_matrix = []
for train, test in kfold.split(inputs, targets):
      # Compile the model
    model = create_model(0.1)
    
    X_train_res, y_train_res = inputs[train], targets[train]
    X_test_res, y_test_res = inputs[test], targets[test]
        
      # Generate a print
    print('-----------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model , class_weight=class_weight
    history = model.fit(X_train_res, y_train_res, class_weight=class_weights, validation_data=(X_test_res, y_test_res),batch_size=256, epochs=100,callbacks=[custom_early_stopping, reduce_lr])

    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    fig, axs = plt.subplots(2)
    fig.suptitle('model loss')
    fig.legend(['train', 'val'], loc='upper left')

    axs[0].plot(history.history['loss'])
    axs[0].plot(history.history['val_loss'])
    axs[1].plot(history.history['auc'])
    axs[1].plot(history.history['val_auc'])

    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

      # Increase fold number
    fold_no = fold_no + 1

In [ ]:
# Predict Neural network
y_pred = model.predict(x_test_pca)
y_1 = np.round(y_pred)
score = roc_auc_score(y_test, y_1)
#score = roc_auc_score(y_test, clf_1.predict_proba(x_test), multi_class='ovr')
print('ROC AUC: %.3f' % score)
print(classification_report_imbalanced(y_test, y_1))
cm = metrics.confusion_matrix(y_test, y_1, normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(cm).plot()